In [3]:
import numpy as np 
import matplotlib.pyplot as plt
import scipy.constants as const
from scipy.signal import convolve
from scipy.fft import rfft, irfft, rfftfreq
from scipy import signal
from scipy.interpolate import interp1d

# 常量定义
G = const.G # 万有引力常数, m^3 kg^-1 s^-2
c = const.c  # 光速, m/s
msun = 1.989e30  # 太阳质量, kg
pc = 3.086e16  # pc到m的转换

In [4]:
# 初始参数 - GW150914
m1 = 36          # 第一个黑洞质量（太阳质量）
m2 = 29          # 第二个黑洞质量（太阳质量）
M_chirp = (m1 * m2 / (m1 + m2) ** 2) ** (3/5) * (m1 + m2)  # 啁啾质量
M_chirp = M_chirp * msun  # 转换为kg

# 透镜参数
mlz = 1e7 * msun  # 透镜质量
y = 2       # 影响参数

# 采样参数 - 与LIGO一致
fs = 16384       # 采样率调整为LIGO的标准采样率
dt = 1/fs
t = np.arange(0, 10, dt)  # 时间范围调整，使其更加聚焦于合并前的最后阶段
t_len = t[-1] - t[0]
N = len(t)
print(f'm1：{m1}')
print(f'm2：{m2}')
print(f'啁啾质量：{M_chirp / msun:.2f}')
print(f'采样率：{fs}')
print(f'总样本数：{N}')
# 引力波参数 - 调整以匹配GW150914
tc = 8      # 合并时间，选择使信号在最后时刻达到峰值
phi_c = 0         # 合并时的相位

# SIS透镜参数
r = 410 * 1e6 * pc   # 源距离 (410 Mpc)


# 函数：引力波波形生成（啁啾信号）
def generate_gw_signal(t):
    # 确保在合并前截止
    valid_idx = t < tc
    t_valid = t[valid_idx]
    # 计算Θ(t)，控制信号的频率演化
    Theta = c**3 * (tc - t_valid) / (5 * G * M_chirp)
    # 计算振幅部分
    A = (G * M_chirp / (c**2 * r)) * Theta**(-1/4)
    
    # 原始相位计算
    phase = 2 * phi_c - 2 * Theta**(5/8)
    phase = 0.4 * phase
    
    # 计算波形
    h = np.zeros_like(t)
    h[valid_idx] = A * np.cos(phase)
    return h

## 函数：透镜放大因子（时域方法2）
def lens_time_domain_advanced(h_original, t, y):
    # 计算放大因子
    mu_plus = np.sqrt(1 + 1/y)  # 主像
    mu_minus = np.sqrt(-1 + 1/y) if y < 1 else 0  # 次像
    
    # 计算物理时间延迟
    td = 2 * y  # 秒
    # 创建插值函数
    interp_func = interp1d(t, h_original, kind='cubic', bounds_error=False, fill_value=0)
    # 创建延迟时间点
    t_delayed = t +td 
    
    # 计算延迟信号
    h_delayed = interp_func(t_delayed)
    
    # 透镜化效应：原始信号和延迟信号的组合
    h_lensed = mu_plus * h_original
    
    # 只有在y<1时才有次像
    if y < 1:
        h_lensed = h_lensed - mu_minus * h_delayed  # 次像有π相位差(负号)
    return h_lensed
# 计算引力波信号
h_original = generate_gw_signal(t)


# 计算放大因子
mu_plus = np.sqrt(np.abs(1 + 1/y))
mu_minus = np.sqrt(np.abs(-1 + 1/y)) if y < 1 else 0


h_lens_new = lens_time_domain_advanced(h_original, t, y)
# 输出计算的时间延迟
print(f"计算的时间延迟: {2 * y} 秒")
print(f"透镜效应中的mu_plus: {mu_plus:.4f}")
print(f"透镜效应中的mu_minus: {mu_minus:.4f}")
print(f'当y>1时的放大倍率：{h_lens_new / h_original:.4f}')
# 绘制原始信号的时域和频域表示
plt.figure(figsize=(18, 12)) 

# 时域原始信号
plt.subplot(221)
plt.plot(t, h_original)
plt.title('Signal (Time Domain)')
plt.xlabel('Time (s)')
plt.ylabel('Strain')
plt.grid(True)

# 透镜后信号（时域）
plt.subplot(222)
plt.plot(t, h_original,label = 'signal')
plt.plot(t, h_lens_new, label='Time Domain method 2')
plt.title('Lensed Signal (Time Domain)')
plt.xlabel('Time (s)')
plt.ylabel('Strain')
plt.legend()
plt.grid(True)
plt.savefig('lensing_effects.png', dpi=300)
plt.subplot(223)
plt.plot(t, h_original, label='Original Signal')
plt.plot(t, h_lens_new, label='Lens Signal(T)')
plt.title('Signal Comparison (Time method 2)')
plt.xlabel('Time (s)')
plt.ylabel('Strain')
plt.legend()
plt.grid(True)
plt.grid(True)
plt.subplot(224)
plt.plot(t, h_lens_new, label='Lens Signal(T)')
plt.title('Signal Comparison (Time method 2)')
plt.xlabel('Time (s)')
plt.ylabel('Strain')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('signal_lens_comparison.png', dpi=300)
plt.show()


m1：36
m2：29
啁啾质量：28.10
采样率：16384
总样本数：163840
计算的时间延迟: 4 秒
透镜效应中的mu_plus: 1.2247
透镜效应中的mu_minus: 0.0000


/tmp/ipykernel_10282/60694393.py:86: RuntimeWarning: invalid value encountered in divide
  print(f'当y>1时的放大倍率：{h_lens_new / h_original:.4f}')


TypeError: unsupported format string passed to numpy.ndarray.__format__